In [2]:
import tweepy
import csv
import json
import pandas as pd
import requests
from bs4 import BeautifulSoup as BS
import re
import time
import numpy as np
import jsonlines
import io

First part of the program where the data will be extracted and will be saved in json file

In [4]:
#loading twitter API credentials from the file which was saved
with open('twitter_credentials.json') as cred_data:
    info = json.load(cred_data)
    consumer_key = info['CONSUMER_KEY']
    consumer_secret = info['CONSUMER_SECRET']
    access_key = info['ACCESS_KEY']
    access_secret = info['ACCESS_SECRET']

In [5]:
# initialization of a list to hold all Tweets
alltweets = []

In [6]:
# Authorization and initialization
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
#Request result parsed for json.
api = tweepy.API(auth,parser=tweepy.parsers.JSONParser())

In [7]:
# We will get the tweets with multiple requests of 200 tweets each, and get it in the extended format.
alltweets = api.search(q="midasIIITD", count=200, tweet_mode='extended')


In [45]:
# Save the GET query in a json format.
with open('midasIIITD.jsonl', 'w') as outfile:
    for entry in alltweets['statuses']:
        json.dump(entry, outfile)
        outfile.write('\n')

In [31]:
n_images = []

In [17]:
len(alltweets['statuses'])

45

In [32]:
## Looping through each tweet to retrieve the amount of media/image files.
for i in range(len(alltweets['statuses'])):
    image = 0
    try:
        for i in alltweets['statuses'][i]['entities']['media']:
            image = image + 1
    except:
        image = None
    
    n_images.append(image)

Second part of the program to parse the json file and display the output in tabular form

In [25]:
rt = []
images = []
text = []
created_at = []
favorite = []

In [26]:
##Using prettytable library to plot table in a friendly way 
for i in range(len(alltweets['statuses'])):
    text.append(alltweets['statuses'][i]['full_text'])
    created_at.append(alltweets['statuses'][i]['created_at'])
    favorite.append(alltweets['statuses'][i]['favorite_count'])
    rt.append(alltweets['statuses'][i]['retweet_count'])

In [35]:
new_dataframe = pd.DataFrame()

In [36]:
new_dataframe['text'] = text

In [37]:
new_dataframe['Date and time'] = created_at

In [38]:
new_dataframe['number of favorites/likes'] = favorite

In [39]:
new_dataframe['retweets count'] = rt

In [41]:
new_dataframe['images'] = n_images

In [42]:
new_dataframe

,text,Date and time,number of favorites/likes,retweets count,images
0,We request all students whose interview are sc...,Sun Apr 07 11:43:24 +0000 2019,0,0,NaN
1,"Other queries: ""none of the Tweeter Apis give ...",Sun Apr 07 06:55:19 +0000 2019,3,2,NaN
2,"Other queries: ""do we have to make two differe...",Sun Apr 07 06:53:38 +0000 2019,3,1,NaN
3,RT @midasIIITD: Response to some queries asked...,Sun Apr 07 06:50:11 +0000 2019,0,1,NaN
4,"Other queries: ""If using Twitter api, it does ...",Sun Apr 07 05:32:27 +0000 2019,4,1,NaN
5,Response to some queries asked by students on ...,Sun Apr 07 05:29:40 +0000 2019,6,1,NaN
6,RT @midasIIITD: @nupur_baghel @PennDATS Congra...,Sat Apr 06 18:42:01 +0000 2019,0,3,NaN
7,RT @midasIIITD: @nupur_baghel @PennDATS Congra...,Sat Apr 06 17:54:43 +0000 2019,0,3,NaN
8,RT @kdnuggets: Top 8 #Free Must-Read #Books on...,Sat Apr 06 17:11:29 +0000 2019,0,2,NaN
9,RT @midasIIITD: @nupur_baghel @PennDATS Congra...,Sat Apr 06 16:44:58 +0000 2019,0,3,NaN


In [43]:
new_dataframe.to_csv('All_tweets_MIDASIIITD.csv')